In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import hamming_loss
import csv
import itertools
from random import shuffle

In [2]:
# Random shuffle of data

link='preprocessed_data.csv'

with open(link, newline='', encoding='utf8') as f:
    data = f.readlines()

header = data[0]
rest = data[1:]
shuffle(rest)

temp_link='temp_preprocessed_data.csv'

print(rest)

with open(temp_link, 'w') as out:
    #out.writelines(data)
    out.write(''.join([header]+rest))

['XNxG2PBVMlI,Louis CK Calls Trump A Gross Crook Dirty Rotten Lying Sack Of,Louis CK regrets comparing Donald Trump to Hitler But hes not going to take it back Subscribe To The Late Show Channel HERE ,45101,5348,,Corden,letterman,The Late Show,Colbert,celeb,comedian,impressions,funny,CBS,skits,david letterman,Important,Late Show,hollywood,comedy,James Corden,celebrities,funny video,Late Late Show,bit,late night,joke,celebrity,talk show,funny videos,jokes,The Late Late Show,Stephen Colbert,monologue\r\n', 'L80Ua3Nh-4Y,Spicer reads to kids at the Easter Egg Roll,Press secretary Sean Spicer read the book How To Catch The Easter Bunny to a group of kids at the White House Easter Egg Roll,119,129,,CNN,News,default,CNN TV,CNN Newsroom\r\n', 'N4bU1i-XAxE,Animal Adventure Park Giraffe Cam,VOTE TO NAME THE BABY WWWNAMEAPRILSCALFCOM TEXT ALERT SYSTEM NOW AVAILABLE wwwAprilthegiraffecom Animal Adventure Park ,81758,10621,,Harpursville,giraffe cam,Park,Zoo,Giraffe,Adventure,Animal,New York,Animal 

In [3]:
# Removing empty rows

shuffle_link='shuffled_preprocessed_data.csv'

input1 = open(temp_link, 'r')
output1 = open(shuffle_link, 'w')

reader = csv.reader(input1)
writer = csv.writer(output1)

for line in input1.readlines():
    if line.strip()=='':
        continue
    output1.write(line)

input1.close()
output1.close()

In [4]:
# Splitting data into training and testing sets

link='shuffled_preprocessed_data.csv'

input1 = open(link, 'r')

lines = itertools.islice(input1,1,None)
reader = csv.reader(lines)

full_len = len(list(reader))
thres = int((full_len*3)/4)

print(thres)

input1.close()
              
with open(link, newline='', encoding='utf8') as f:
    cnt=0
    X_arr=[]
    X_test_arr=[]
    
    y_train_text=[]
    Y_test=[]
    
    lines = itertools.islice(f,1,None)
    reader = csv.reader(lines)
    
    for row in reader:
        if(cnt<thres):
            X_arr.append(row[1]+', '+row[2])
            list1=[]
            for i in range(6,len(row)):
                list1.append(row[i])
                
            y_train_text.append(list1)
        else:
            X_test_arr.append(row[1]+', '+row[2])
            list1=[]
            for i in range(6,len(row)):
                list1.append(row[i])
            Y_test.append(list1)
        
        cnt=cnt+1
        
    X_train = np.array(X_arr)
    X_test = np.array(X_test_arr)
    print(len(Y_test))

21
8


In [5]:
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(y_train_text)

In [6]:
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])

In [7]:
classifier.fit(X_train, Y)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...lti_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1))])

In [8]:
predicted = classifier.predict(X_test)
#print(predicted)

In [9]:
text_labels = mlb.inverse_transform(predicted)

In [10]:
'''
train_data=[]
for test_text, predict_labels in zip(X_test, text_labels):
    #print('{0} => {1}'.format(test_text, ', '.join(predict_labels)))
    #if(predict_labels!=''):
    if predict_labels:
        train_data.append(test_text)
#print(len(train_data))
'''

# Storing all the predicted labels

test_data=[]
all_predicted_labels=[]
#temp_list=[]

for actual_text, predict_labels in zip(Y_test, text_labels):
    #print('{0} => {1}'.format(actual_text, ', '.join(predict_labels)))
    print(predict_labels)
    
    all_predicted_labels.append(predict_labels)
    
    if predict_labels:
        #print(predict_labels)
        #cnt_none+=1
        test_data.append(actual_text)
print(len(test_data))

'''
test_predicted_data=[]
for actual_text, predict_labels in zip(predicted, text_labels):
    #print('{0} => {1}'.format(actual_text, ', '.join(predict_labels)))
    #print(actual_text)
    if predict_labels:
        cnt_none+=1
        test_predicted_data.append(actual_text)
#print(test_predicted_data[0])
'''

()
('YouTube Editor', 'street vehicle toys', 'wooden educational toys')
('CNN', 'CNN Newsroom', 'CNN TV', 'News', 'default')
()
()
()
('African Elephant Animal ', 'Djuma Game Reserve Location ', 'Giraffe Animal ', 'Kruger National Park Protected Site ', 'Leopard Animal ', 'Lion Animal ', 'Livestream Website ', 'Safari Industry ', 'Wildlife Film Subject ')
('African Elephant Animal ', 'Djuma Game Reserve Location ', 'Giraffe Animal ', 'Kruger National Park Protected Site ', 'Leopard Animal ', 'Lion Animal ', 'Livestream Website ', 'Safari Industry ', 'Wildlife Film Subject ')
4


"\ntest_predicted_data=[]\nfor actual_text, predict_labels in zip(predicted, text_labels):\n    #print('{0} => {1}'.format(actual_text, ', '.join(predict_labels)))\n    #print(actual_text)\n    if predict_labels:\n        cnt_none+=1\n        test_predicted_data.append(actual_text)\n#print(test_predicted_data[0])\n"

In [11]:
Y_new = mlb.fit_transform(Y_test)
#print(Y_new)
#Y_new = mlb.fit_transform(test_data)

In [12]:
read_link='shuffled_preprocessed_data.csv'

# Header
list1=['Id', 'Title', 'Description', 'LikeCount', 'DislikeCount', 'Location (latitude, longitude)', 'Condensed Tags (comma delimited string)']
tot_list=[]
tot_list.append(list1)

with open(read_link, newline='', encoding='utf8') as f:
    lines = itertools.islice(f,1+thres,None)
    reader = csv.reader(lines)
    
    row_num=0
    for row in reader:
        list2=[]
        for i in range(0,6):
            list2.append(row[i])
        list2.extend(all_predicted_labels[row_num])
        tot_list.append(list2)
        row_num+=1
    print(tot_list)

[['Id', 'Title', 'Description', 'LikeCount', 'DislikeCount', 'Location (latitude, longitude)', 'Condensed Tags (comma delimited string)'], ['Ae80m8Efk7M', 'Russia vetoes UN Syrian resolution', 'Russia vetoes the United Nations resolution requiring Syria to cooperate with investigators after a brutal chemical attack', '147', '67', ''], ['zUQJX1jkZGE', 'Learn Colors with Wodden Hammer Educational Toys  Colors Videos Collection for Children', 'Learn Colors with Wooden Hammer Educational Toys  Colors Videos Collection for Children 0000  Learning Colors Street Vehicles Names and Sounds for ', '0', '0', '', 'YouTube Editor', 'street vehicle toys', 'wooden educational toys'], ['lWnrnCg6xGc', 'Police Ohio murder suspect killed himself', 'Steve Stephens the Cleveland murder suspect who posted video of the slaying on Facebook killed himself in Pennsylvania after a police chase officers said ', '428', '73', '', 'CNN', 'CNN Newsroom', 'CNN TV', 'News', 'default'], ['4EOrflNS6Wg', 'Syrian Chemical 

In [13]:
# Writing to file the predicted labels

write_link='predicted_data.csv'

with open(write_link, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(tot_list)

In [14]:
#print("Accuracy = " + str(1-hamming_loss(np.array(predicted),np.array(Y_new))))
#print("Accuracy = " + str(1-hamming_loss(np.array(test_predicted_data),np.array(Y_new))))
#print(len(predicted[0]))
#print(len(Y_new[0]))

In [15]:
#acc = classifier.score(X_test,Y_new)

In [16]:
acc=0
for i in range(0,len(Y_test)):
    acc+=(len(set(Y_test[i]) & set(text_labels[i])))/(len(set(Y_test[i])))

In [17]:
print(acc)
real_acc=acc/len(Y_test)
curr_acc=acc/len(test_data)

3.8333333333333335


In [18]:
# Accuracy measures

# First accuracy is the actual accuracy of all labels

# Second accuracy is only for the those labels 
# which rendered at least one label as output

print(real_acc)
print(curr_acc)

0.4791666666666667
0.9583333333333334
